# "The disturbed outer Milky Way disc", McMillan et al (2022)
# Create plots with mock data

### Package requirements: numpy, scipy, pandas, agama

# Version history

v1 - Copied across from my ugly code to make public with initial arXiv publication

In [ ]:

import agama,  sys, os
import numpy as np
from configparser import RawConfigParser  # python 3
from plotparameters import *
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import scipy.stats

MockDataDir = f'MockData/'


LZlabel='$L_Z^*$ [kpc$\,$km$\,$s$^{-1}$]'
mycolormap = generatecmap('plasma_r')
RGlabel='$R_g^*$ [kpc]'

### Setup AGAMA distribution function

In [ ]:

iniFileName = "AgamaFiles/SimpleDiscs.ini"

ini = RawConfigParser()
ini.optionxform=str  # do not convert key to lowercase
ini.read(iniFileName)

iniDFThinDiskRs  = dict(ini.items("DF thin disk high Rsig"))
iniDFThickDisk   = dict(ini.items("DF thick disk"))
iniDFStellarHalo = dict(ini.items("DF stellar halo"))


# define external unit system describing the data (including the parameters in INI file)
agama.setUnits(length=1, velocity=1, mass=1)   # in Kpc, km/s, Msun


pot = agama.Potential("AgamaFiles/McMillan17.ini") 
dfThinDisk    = agama.DistributionFunction(potential=pot, **iniDFThinDiskRs)
dfThickDisk   = agama.DistributionFunction(potential=pot, **iniDFThickDisk)
dfStellarHalo = agama.DistributionFunction(potential=pot, **iniDFStellarHalo)
# composite DF of all stellar components except the bulge
dfStellar     = agama.DistributionFunction(dfThinDisk, dfThickDisk, dfStellarHalo)

actf= agama.ActionFinder(pot)

#### Print out parameters for LaTeX table

In [ ]:
for k in iniDFThinDiskRs.keys():
    print(k + ' & ',end='')
print()

for k in iniDFThinDiskRs.keys():
    print(iniDFThinDiskRs[k] + ' & ',end='')
print()

for k in iniDFThickDisk.keys():
    #print(k + ' & ',end='')
    print(iniDFThickDisk[k] + ' & ',end='')
print(' & &')

for k in iniDFStellarHalo.keys():
    print(k + ' & ',end='')
print()

for k in iniDFStellarHalo.keys():
    print(iniDFStellarHalo[k] + ' & ',end='')
print()


In [ ]:
mod = agama.GalaxyModel(potential=pot, df=dfStellar, af=actf)

## Perturbations in vz & vR

In [ ]:

def vzAdjustment(Lz) :
    '''Perturb vertical velocity to mimic that seen in Galactic anticentre.'''
    dvz = np.zeros_like(Lz)
    mask1 = Lz<2000
    mask2 = (Lz>=2000) & (Lz<2250)
    mask3 = (Lz>=2250) & (Lz<2750)
    mask4 = Lz>=2750
    dvz[mask1] = 0.
    dvz[mask2] = 6.*(Lz[mask2]-2000.)/250.
    dvz[mask3] = 6.- 20*(Lz[mask3]-2250.)/500.
    dvz[mask4] = 5. 
    return dvz

def simpleSpiralModel(R, phi, vR_amplitude=7., m=2, phi0=0, pitch=12) : 
    '''Perturb Galactocentric vR to mimic spiral arms effect'''
    return vR_amplitude*np.sin(m*(phi - phi0 - np.log(R)/np.tan(np.deg2rad(pitch))))



### Functions to
1. Convert cartesian to cylindrical polar
2. Convert cylindrical polar to cartesian
3. Convert Galactocentric x,y,z,vx,vy,vz to Heliocentric r,l,b,vlos,mul,mub
4. Convert Heliocentric r,l,b,vlos,mul,mub to Galactocentric R, phi, Z, vR, vphi, vZ...
    * using all velocity components
    * without vlos and assuming that vR = 0
    

In [ ]:

def Cart2Cyl(x,y,z,vx,vy,vz) :
    '''Convert cartesian position & velocity to cylindrical polars'''
    R   = np.sqrt(x**2+ y**2)
    phi = np.arctan2(y,x)
    vR  = (x*vx+y*vy)/R
    vphi = (x*vy-vx*y)/R
    return R,phi,z,vR,vphi,vz

def Cyl2Cart(R,phi,z,vR,vphi,vz) :
    '''Convert cylindrical polar position & velocity to cartesian'''
    x = R*np.cos(phi)
    y = R*np.sin(phi)
    vx= vR*np.cos(phi) - vphi*np.sin(phi)
    vy= vR*np.sin(phi) + vphi*np.cos(phi)
    return x,y,z,vx,vy,vz



def CartG2SphH(xG,yG,zG,vxG,vyG,vzG,Rsun=8.21,zsun=0.02, Usun=11.1, Vsun=248.5, Wsun=7.25) :
    '''Convert from Galactocentric Cartesian to Heliocentric Galactic coordinates. Lazy unit conversion for pm.'''
    # Heliocentric 
    x,y,z,vx,vy,vz = Rsun-xG,-yG,zG-zsun,-(vxG+Usun),-(vyG+Vsun),vzG-Wsun
    
    R = np.sqrt(x**2+y**2)
    r = np.sqrt(x**2 + y**2 + z**2)
    cosl,sinl,cosb,sinb = x/R,y/R,R/r,z/r
    temp = cosl*vx + sinl*vy
    l = np.rad2deg(np.arccos(cosl))
    l[sinl<0.] = 360-np.rad2deg(np.arccos(cosl[sinl<0.]))
    b = np.rad2deg(np.arcsin(sinb))
    vr = cosb*temp + sinb*vz
    mul = (cosl*vy - sinl*vx) / r;
    mub = (cosb*vz - sinb*temp  ) / r;
    return r,l,b,vr,mul,mub

def SphH2CylG(r,l,b,vr,mul,mub,Rsun=8.21,zsun=0.02, Usun=11.1, Vsun=248.5, Wsun=7.25) :
    '''Convert from Heliocentric Galactic coordinates to Galactocentric Cartesian. If vr is unknown, estimate it.'''
    cl,sl,cb,sb = np.cos(np.deg2rad(l)),np.sin(np.deg2rad(l)),np.cos(np.deg2rad(b)),np.sin(np.deg2rad(b))
    # Fix the below
    Xh  = cl*cb*r
    Yh  = sl*cb*r
    Zh  = sb*r
    vl  = mul*r
    vb  = mub*r
    vlos = []
    if vr is None :
        vlos = np.zeros_like(mul)
    else :
        vlos = vr[:]
        
    VXh =  vl*(-sl) + vb*(-cl*sb) + vlos*cl*cb
    VYh =  vl*(cl) + vb*(-sl*sb)  + vlos*sl*cb
    VZh =  vb*(cb) + vlos*sb
    # Now flip my system like a lunatic
    Xg = Rsun-Xh
    Yg = -Yh
    Zg = Zh+zsun
    VXg = -(VXh + Usun)
    VYg = -(VYh + Vsun)
    VZg =  (VZh + Wsun)
    Rg,PHIg,Zg,VRg,VPHIg,VZg = Cart2Cyl(Xg,Yg,Zg,VXg,VYg,VZg)
    if vr is None :
        Vlos_guess = (Xg*VXg + Yg*VYg)/(Xg*cl*cb+Yg*sl*cb)
        Vphi_guess =  Rg*(-sl*VXg + cl*VYg)/(Xg*cl + Yg*sl)
        Vz_guess = VZg + Vlos_guess*sb
        VRg,VPHIg,VZg = np.zeros_like(VRg),Vphi_guess,Vz_guess
    return Rg,PHIg,Zg,VRg,VPHIg,VZg


## Create (or read in) the sample from agama df, and add perturbation in vz

In [ ]:

nSample = 40000000 # Full sample size (includes all radii, so useful sample that is saved is much smaller)
R0 = 8.21          # Solar position 
Vc = 236.          # Circular velocity for conversion to guiding radius

sampleFile = MockDataDir + f'FilteredSampleN{nSample}.csv'

if not os.path.exists(sampleFile):
    print('Creating new sample data...')
    # Create sample
    xv_ini, _ = mod.sample(nSample)
    R = np.sqrt(xv_ini[:,0]**2+ xv_ini[:,1]**2)
    # Cut stars at z which will never enter |b|<10 sample
    coslmin = np.cos(np.deg2rad(130))
    zmax = np.arctan(np.deg2rad(10.))*(np.sqrt((R0*coslmin)**2 + R**2 - R0**2) + R0*coslmin)
    mask = (R>R0) & (np.abs(xv_ini[:,2]) < zmax) & (R<18.)
    xv = xv_ini[mask]
    df = pd.DataFrame(data={'x': xv[:,0], 'y': xv[:,1], 'z' : xv[:,2], 
                            'vx': xv[:,3],'vy' : xv[:,4], 'vz' : xv[:,5]})
    df.to_csv(sampleFile)
else:
    # Simplest to read in using np
    print('Using existing data...')
    xv = np.loadtxt(sampleFile,delimiter=',',skiprows=1, usecols=[1,2,3,4,5,6])
xv_ini = []



R,phi,z,vR,vphi,vz = Cart2Cyl(xv[:,0],xv[:,1],xv[:,2],xv[:,3],xv[:,4],xv[:,5])

# AGAMA disc df has positive sense of rotation. Milky Way has negative.
# This is a lazy person's way to have the opposite J_phi sign dependance of the df
vphi = -vphi # Sign convention issue

AMtest = pd.DataFrame({'R' : R, 'phi' : phi, 'z' : z,
                       'vR' : vR, 'vphi' : vphi, 'vz' : vz+vzAdjustment(-R*vphi)})
tmp = Cyl2Cart(AMtest.R,AMtest.phi,AMtest.z,AMtest.vR,AMtest.vphi,AMtest.vz)
AMtest['x'],AMtest['y'],AMtest['z'],AMtest['vx'],AMtest['vy'],AMtest['vz'] = tmp

#x,y,z,vx,vy,vz = Cyl2Cart(R,phi,z,vR,vphi,vz)

## Model true vZ for different Lz

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,3)) 

nLz,nVZ,nLz_w,nVZ_w = 51,51,41,31
Lzbins = np.linspace(-3500,-2000,nLz)
VZbins = np.linspace(-50,50,nVZ)

VZbinwidth = VZbins[1]-VZbins[0]

H,xedge,yedge = np.histogram2d(AMtest.R*AMtest.vphi,
                               AMtest.vz,
                               bins=[Lzbins,VZbins])

H1,x1 = np.histogram(AMtest.R*AMtest.vphi,
                    Lzbins)


for i in range(len(H1)) :
    H[i,:] /= H1[i]*VZbinwidth
    
mask = np.abs(AMtest.vz.values) < 50
vmax = 0.03* 20./np.std(AMtest.vz.values[mask]) # ad hoc rescaling

plt.pcolormesh(Lzbins,VZbins,H.T,vmax=vmax,cmap=mycolormap,rasterized=True)
ax.imshow(H.T,origin='lower',extent=[Lzbins[0],Lzbins[-1],-50,50],
                            aspect=10,vmax=vmax,cmap=mycolormap)

ymin,ymax = ax.get_ylim()
ax.vlines([-2500,-3000],ymin=ymin,ymax=ymax,ls=':',color='k',alpha=0.5)

# Axis labels not starred because true values
ax.set_xlabel('$L_Z$ [kpc\,km$\,$s$^{-1}$]')
ax.set_ylabel('$V_Z$ [km$\,$s$^{-1}$]')
ax.invert_xaxis()
secax1 = ax.secondary_xaxis('top', functions=(lambda x: -x/Vc, lambda x: -x*Vc))
ax.tick_params(axis='x', which='both', top=False)
secax1.tick_params(axis='x',which='both',top=True,direction='out')
# Not starred
secax1.set_xlabel('$R_g$ [kpc]',labelpad=5)
plt.savefig(MockDataDir+f'LzVz_model.png',  format='png', dpi=300,bbox_inches='tight')
plt.savefig(MockDataDir+f'LzVz_model.pdf',  format='pdf', dpi=300,bbox_inches='tight')

plt.show()

### Plot Vr perturbation from spiral arms

In [ ]:
fig, ax = plt.subplots(figsize=(6,4)) # for one-column plots
xRange = np.linspace(-15,15,1000)
yRange = np.linspace(-15,15,1000)
XX,YY = np.meshgrid(xRange,yRange,indexing='ij')
RR,PHI = np.sqrt(XX**2+YY**2),np.arctan2(YY,XX)
dV = simpleSpiralModel(RR,PHI)
dV[(RR<5) | (RR>14)] = np.nan
plt.pcolormesh(XX,YY,dV,shading='nearest',cmap='RdYlBu_r')
plt.xlabel(r'$X$ [kpc]')
plt.ylabel(r'$Y$ [kpc]')
plt.colorbar(label=r'$\Delta V_R$ [km$\,$s$^{-1}$]')
plt.plot(R0, 0, 'kx')
plt.gca().set_aspect('equal')
plt.tight_layout()
plt.savefig(MockDataDir + f'SpiralVelocity.png',  
            format='png', dpi=300,bbox_inches='tight')
plt.show()

### Function to 'observe' the model with distance error and option to use vlos or not

In [ ]:
def ObserveModel(R,phi,z,vR,vphi,vz,DistanceFractionalError=0.1,useVR=False) :
        '''Returns fake 'observed' Galacocentric position and velocity with distance error & vR used/not used
        
        Parameters
        ----------
        Self explanatory.
        useVR: bool (default false)
            If False, stars are approximated as having vR=0 when 'observing'
        
        Returns
        -------
        Observed position & velocity R, phi, z, vR, vphi, vz
        '''
        tmp = Cyl2Cart(R,phi,z,vR,vphi,vz)
        _x,_y,_z,_vx,_vy,_vz = tmp
        # Observed values
        _r,_l,_b,_vr,_mul,_mub = CartG2SphH(_x,_y,_z,_vx,_vy,_vz)
        distchange = 1.+DistanceFractionalError*np.random.randn(len(_r))
        
        _r_obs = _r * distchange
        _r_obs[_r_obs<0.] = _r[_r_obs<0.]
        
        # estimate ignoring vr
        if useVR:
            R_obs,phi_obs,z_obs,vR_obs,vphi_obs,vz_obs = SphH2CylG(_r_obs,_l,_b,_vr,_mul,_mub)
        else :
            R_obs,phi_obs,z_obs,vR_obs,vphi_obs,vz_obs = SphH2CylG(_r_obs,_l,_b,None,_mul,_mub)
        return R_obs,phi_obs,z_obs,vR_obs,vphi_obs,vz_obs



In [ ]:
Lzbins = np.linspace(-3490,-2010,nLz)

def makeLzVzPlot(df_in,DistanceFractionalError=0.15,
                 useVR=False,Spiral=True,fig_y=4.2,
                 labelMiddle=False) :
    '''Makes a 2 by 5 grid of plots of the Vz distribution as a function of Lz, each panel represents a 10 degree range in l'''
    fig, ax = plt.subplots(2,5,figsize=(13,fig_y))
    
    # adjust labels
    if useVR :
        labelLZ = '$L_Z$ [kpc\,km$\,$s$^{-1}$]'
        labelRG = '$R_g$ [kpc]'
        labelVZ = '$V_Z$ [km$\,$s$^{-1}$]'
    else:
        labelLZ, labelRG, labelVZ = LZlabel, RGlabel, VZlabel
    
    for ii in range(10) :
        # Which panel?
        row = ii//5
        itmp = ii%5
        # Which longitude range?
        lmin = 130+10*ii
        lmax = 130+10*(ii+1)
        # Adjust phi values
        phimin = np.deg2rad(180-lmin)-np.arcsin(np.sin(np.deg2rad(lmin))*R0/df_in.R)
        phimax = np.deg2rad(180-lmax)-np.arcsin(np.sin(np.deg2rad(lmax))*R0/df_in.R)
        _phi = np.mod(df_in.phi-phimin,(phimax-phimin))+phimin
        # filter to |b| < 10 
        zmax = np.arctan(np.deg2rad(10.))*np.sqrt(R0**2 + df_in.R.values**2 - 2*R0*df_in.R.values*np.cos(_phi))
        mask_local = (np.abs(df_in.z.values) < zmax) 
        df_use = df_in[mask_local]
        _phi = _phi[mask_local]

        if Spiral:
            _vR = simpleSpiralModel(df_use.R.values,_phi) + df_use.vR
        else:
            _vR = df_use.vR
        
        
        # Make fake observation
        R_obs,phi_obs,z_obs,vR_obs,vphi_obs,vz_obs = ObserveModel(df_use.R, _phi, df_use.z,
                                                                  _vR, df_use.vphi, df_use.vz,
                                                                  DistanceFractionalError=DistanceFractionalError,
                                                                  useVR=useVR)
        
        Lz_tmp = R_obs*vphi_obs
        
        H,xedge,yedge = np.histogram2d(Lz_tmp,
                                       vz_obs,
                                       bins=[Lzbins,VZbins])

        H1,x1 = np.histogram(Lz_tmp, Lzbins)

        for i in range(len(H1)) :
            H[i,:] /= H1[i]*VZbinwidth
            
        vmax = 0.06* 20./np.std(vz_obs) # ad hoc rescaling
        im = ax[row,itmp].imshow(H.T,origin='lower',extent=[Lzbins[0],Lzbins[-1],-50,50],aspect=10,vmax=vmax,
                                 cmap=mycolormap)
        
        ymin,ymax = ax[row,itmp].get_ylim()
        ax[row,itmp].vlines([-2500,-3000],ymin=ymin,ymax=ymax,ls=':',color='k',alpha=0.5)
        
        vz_minmax = np.percentile(vz_obs-df_use.vz,[16,84])
        lz_minmax = np.percentile(df_use.R*df_use.vphi-Lz_tmp,[16,84])
        
        ax[row,itmp].errorbar(-3100,35,0.5*(vz_minmax[1]-vz_minmax[0]), 
                    0.5*(lz_minmax[1]-lz_minmax[0]),color='k',lw=1) ## Use percentiles
        
        ax[row,itmp].invert_xaxis()
        secax1 = ax[row,itmp].secondary_xaxis('top', functions=(lambda x: -x/Vc, lambda x: -x*Vc))
        ax[row,itmp].tick_params(axis='x', which='both', top=False) 
        if row==0:
            secax1.set_xlabel(labelRG,labelpad=5)
            if labelMiddle:
                ax[row,itmp].set_xlabel(labelLZ)
            else:
                ax[row,itmp].tick_params(labelbottom=False)
            ax[row,itmp].set_title(r'$%d<\ell<%d$' % (lmin,lmax),pad=10)
        else:
            if labelMiddle:
                secax1.set_xlabel(labelRG,labelpad=5)
            else:
                secax1.tick_params(labeltop=False)
            ax[row,itmp].set_xlabel(labelLZ)
            ax[row,itmp].set_title(r'$%d<\ell<%d$' % (lmin,lmax))
        if itmp==0:
            ax[row,itmp].set_ylabel(labelVZ)
            ax[row,itmp].tick_params(axis='y', which='major', pad=5)
        else:
            ax[row,itmp].tick_params(labelleft=False)
        
        
        
        ii += 1
    return im, fig, ax

## Everything included. Distance errors, no los velocity, spiral perturbation.

In [ ]:
DistancePercentageError = 15
im, fig, ax = makeLzVzPlot(AMtest,DistancePercentageError/100.,fig_y=4.4)
plt.tight_layout(h_pad=1.2)
fig.colorbar(im, ax=ax.ravel().tolist(),fraction=0.1,pad=0.02,label=r'$P(V_Z^*)$ [km$^{-1}\,$s]',shrink=0.95)

plt.savefig(MockDataDir + f'LzVz_full_{DistancePercentageError}disterr.png', 
            format='png', dpi=100,bbox_inches='tight')
plt.savefig(MockDataDir + f'LzVz_full_{DistancePercentageError}disterr.pdf', 
            format='pdf', dpi=300,bbox_inches='tight')
plt.show()

## Distance errors, but los velocity known and no spiral perturbation.

In [ ]:
Lzbins = np.linspace(-3490,-2010,nLz)
DistancePercentageError = 15
im,fig,ax = makeLzVzPlot(AMtest,DistancePercentageError/100.,useVR=True,Spiral=False,fig_y=4.9)
plt.suptitle('Distance Error Only',size='xx-large')
plt.tight_layout(h_pad=1.5)
###### N.B. make sure filename matches uncertainty
fig.colorbar(im, ax=ax.ravel().tolist(),fraction=0.1,pad=0.02,label=r'$P(V_Z)$ [km$^{-1}\,$s]',shrink=0.9)
plt.savefig(MockDataDir + f'LzVz_DistOnly_{DistancePercentageError}disterr.png', 
            format='png', dpi=100,bbox_inches='tight')
plt.savefig(MockDataDir + f'LzVz_DistOnly_{DistancePercentageError}disterr.pdf', 
            format='pdf', dpi=300,bbox_inches='tight')
plt.show()

## No los velocity, but no distance errors or spiral perturbation.

In [ ]:
Lzbins = np.linspace(-3490,-2010,nLz)
DistancePercentageError = 0.
im, fig, ax = makeLzVzPlot(AMtest,DistancePercentageError/100.,useVR=False,Spiral=False,fig_y=4.9)
plt.suptitle('No LOS velocity only',size='xx-large')
plt.tight_layout(h_pad=1.5)
###### N.B. make sure filename matches uncertainty
fig.colorbar(im, ax=ax.ravel().tolist(),fraction=0.1,pad=0.02,label=r'$P(V_Z^*)$ [km$^{-1}\,$s]',shrink=0.9)
###### N.B. make sure filename matches uncertainty
plt.savefig(MockDataDir + f'LzVz_Vlosonly.png', 
            format='png', dpi=300,bbox_inches='tight')
plt.savefig(MockDataDir + f'LzVz_Vlosonly.pdf', 
            format='pdf', dpi=300,bbox_inches='tight')
plt.show()

## Spiral perturbation & no los velocity, but no distance errors.

In [ ]:
Lzbins = np.linspace(-3490,-2010,nLz)
DistancePercentageError = 0.
im, fig, ax = makeLzVzPlot(AMtest,DistancePercentageError/100.,useVR=False,Spiral=True,fig_y=4.9)
plt.suptitle('Spiral perturbation, no LOS velocity',size='xx-large')
plt.tight_layout(h_pad=1.5)
###### N.B. make sure filename matches uncertainty
fig.colorbar(im, ax=ax.ravel().tolist(),fraction=0.1,pad=0.02,label=r'$P(V_Z^*)$ [km$^{-1}\,$s]',shrink=0.9)
###### N.B. make sure filename matches uncertainty
plt.savefig(MockDataDir + f'LzVz_VlosSpiral.png', 
            format='png', dpi=300,bbox_inches='tight')
plt.savefig(MockDataDir + f'LzVz_VlosSpiral.pdf', 
            format='pdf', dpi=300,bbox_inches='tight')
plt.show()